In [9]:
from utils import encode_image_to_base64
from openai import OpenAI
import fitz  
from pathlib import Path

In [3]:

def analyze_image_gpt4o(prompt, image_path):
    """
    Analyze an image using the GPT-4o model and return a description.

    :param prompt: The text prompt for the model
    :param image_path: Local path to the image file
    :return: Model-generated description or error message
    """
    client = OpenAI()
    
    # Encode the image
    base64_image = encode_image_to_base64(image_path)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Ensure you have set your OpenAI API key as an environment variable
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

result = analyze_image_gpt4o("Describe this image in detail.", "./data/test_images/屏幕截图 2024-10-19 164116.png")
print(result)

The image is a slide titled "CS 188: Artificial Intelligence" with the subtitle "Introduction" centered at the top. Below the title, there's an illustration depicting a robot and a person engaged in a game of chess. The robot, with antennae and a mechanical arm, is on the left side, pointing towards the chessboard. The person, on the right side, has curly hair and is deep in thought, resting their chin on their hand. 

Text beneath the illustration states "Fall 2024" followed by the names "Pieter Abbeel & Igor Mordatch" and "University of California, Berkeley." At the bottom, there is a note in square brackets: "[Many of these slides were originated by Dan Klein and Pieter Abbeel]".


In [7]:
def convert_pdf_to_images(pdf_path, output_dir):
    """
    Convert each page of a PDF file to an image and save them to the specified output directory.

    :param pdf_path: Path to the PDF file
    :param output_dir: Directory to save the converted images
    """
    # Create the output directory if it doesn't exist
    pdf_name = Path(pdf_path).stem
    image_dir = Path(output_dir) / pdf_name
    image_dir.mkdir(parents=True, exist_ok=True)

    # Open the PDF
    doc = fitz.open(pdf_path)

    # Iterate through each page
    for page_num, page in enumerate(doc):
        # Convert the page to an image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # 300 DPI
        
        # Save the image
        image_filename = f"page_{page_num+1}.png"
        pix.save(image_dir / image_filename)

In [8]:
# Example usage
pdf_path = "./data/test_pdfs/cs188-fa24-lec01.pdf"
output_dir = "./data/test_images"
convert_pdf_to_images(pdf_path, output_dir)


In [11]:
import cv2
import numpy as np
from pathlib import Path
import os

def calculate_similarity(img1, img2):
    """
    Calculate the similarity between two images using ORB feature matching.
    This method is invariant to translation and rotation.
    
    :param img1: First image
    :param img2: Second image
    :return: A similarity score between 0 and 1
    """
    # Initialize ORB detector
    orb = cv2.ORB_create()
    
    # Find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Match descriptors
    matches = bf.match(des1, des2)
    
    # Sort them in the order of their distance
    matches = sorted(matches, key=lambda x: x.distance)
    
    # Calculate similarity score
    similarity = len(matches) / max(len(kp1), len(kp2))
    
    return similarity

def merge_similar_images(image_dir, output_dir, similarity_threshold=0.7):
    """
    Merge similar consecutive images in a directory while maintaining the original order.
    
    :param image_dir: Directory containing the images
    :param output_dir: Directory to save the merged images
    :param similarity_threshold: Threshold for considering images as similar
    """
    # Ensure output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get all image files sorted by name
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
    
    merged_groups = []
    current_group = [image_files[0]]
    
    for i in range(len(image_files) - 1):
        img1 = cv2.imread(os.path.join(image_dir, image_files[i]))
        img2 = cv2.imread(os.path.join(image_dir, image_files[i+1]))
        
        similarity = calculate_similarity(img1, img2)
        
        if similarity >= similarity_threshold:
            current_group.append(image_files[i+1])
        else:
            merged_groups.append(current_group)
            current_group = [image_files[i+1]]
    
    # Add the last group
    if current_group:
        merged_groups.append(current_group)
    
    # Merge and save images
    for i, group in enumerate(merged_groups):
        if len(group) == 1:
            img = cv2.imread(os.path.join(image_dir, group[0]))
            merged = img
        else:
            images = [cv2.imread(os.path.join(image_dir, f)) for f in group]
            heights = [img.shape[0] for img in images]
            max_width = max(img.shape[1] for img in images)
            merged = np.vstack([cv2.resize(img, (max_width, img.shape[0])) for img in images])
        
        # Use the first image's number in the group for naming
        first_num = int(group[0].split('_')[1].split('.')[0])
        cv2.imwrite(os.path.join(output_dir, f'merged_{first_num:03d}.png'), merged)
    
    print(f"Merged images saved to {output_dir}")

# Example usage
pdf_name = "cs188-fa24-lec01"
image_dir = f"./data/test_images/{pdf_name}"
output_dir = f"./data/test_images/{pdf_name}_merged"
merge_similar_images(image_dir, output_dir, similarity_threshold=0.7)

Merged images saved to ./data/test_images/cs188-fa24-lec01_merged
